In [2]:
import numpy as np
import pandas as pd

In [3]:
df=pd.read_csv('IMDB Dataset.csv')

In [4]:
df.shape

(14911, 2)

In [5]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [6]:
y=df['sentiment']
y

0        positive
1        positive
2        positive
3        negative
4        positive
           ...   
14906    negative
14907    negative
14908    negative
14909    negative
14910    positive
Name: sentiment, Length: 14911, dtype: object

In [7]:
y=pd.get_dummies(y)
y=y.iloc[:,1].values

In [8]:
import re
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [9]:
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
lemmatizer=WordNetLemmatizer()

In [10]:
corpus=[]
for i in range(len(df['review'])):
    sent=re.sub('[^a-zA-Z]'," ",df['review'][i])
    sent=sent.lower()
    sent = sent.split()
    
    sent = [lemmatizer.lemmatize(word) for word in sent if not sent in stopwords.words('english')]
    sent = ' '.join(sent)
    corpus.append(sent)
    
    

In [11]:
corpus


['one of the other reviewer ha mentioned that after watching just oz episode you ll be hooked they are right a this is exactly what happened with me br br the first thing that struck me about oz wa it brutality and unflinching scene of violence which set in right from the word go trust me this is not a show for the faint hearted or timid this show pull no punch with regard to drug sex or violence it is hardcore in the classic use of the word br br it is called oz a that is the nickname given to the oswald maximum security state penitentary it focus mainly on emerald city an experimental section of the prison where all the cell have glass front and face inwards so privacy is not high on the agenda em city is home to many aryan muslim gangsta latino christian italian irish and more so scuffle death stare dodgy dealing and shady agreement are never far away br br i would say the main appeal of the show is due to the fact that it go where other show wouldn t dare forget pretty picture pain

In [12]:
print(type(corpus))

<class 'list'>


In [13]:
words=[]
for i in corpus:
    word=i.split()
    words.append(word)
    
print(words)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [14]:
import gensim

In [15]:
model=gensim.models.Word2Vec(words,window=5,min_count=2)

In [16]:
model.wv.index_to_key

['the',
 'a',
 'and',
 'of',
 'to',
 'is',
 'it',
 'br',
 'in',
 'i',
 'this',
 'that',
 's',
 'movie',
 'wa',
 'film',
 'with',
 'for',
 'but',
 'you',
 'on',
 't',
 'not',
 'are',
 'he',
 'his',
 'have',
 'one',
 'be',
 'all',
 'at',
 'they',
 'by',
 'an',
 'who',
 'like',
 'so',
 'from',
 'there',
 'or',
 'just',
 'her',
 'out',
 'about',
 'if',
 'ha',
 'what',
 'time',
 'some',
 'good',
 'can',
 'character',
 'more',
 'when',
 'very',
 'she',
 'up',
 'no',
 'story',
 'get',
 'would',
 'even',
 'my',
 'make',
 'see',
 'which',
 'really',
 'their',
 'only',
 'had',
 'scene',
 'me',
 'well',
 'were',
 'we',
 'than',
 'much',
 'bad',
 'other',
 'been',
 'people',
 'great',
 'into',
 'do',
 'because',
 'will',
 'show',
 'first',
 'also',
 'how',
 'don',
 'most',
 'way',
 'him',
 'thing',
 'made',
 'then',
 'them',
 'too',
 'could',
 'life',
 'any',
 'think',
 'after',
 'go',
 'know',
 'watch',
 'actor',
 'love',
 'two',
 'plot',
 'being',
 'many',
 'year',
 'never',
 'seen',
 'acting',


In [17]:
model.corpus_count

14911

In [18]:
model.wv.similar_by_word('actor')

[('cast', 0.7545036673545837),
 ('actress', 0.7196708917617798),
 ('performer', 0.7187697887420654),
 ('player', 0.668758749961853),
 ('choice', 0.6060278415679932),
 ('performance', 0.6050801277160645),
 ('talent', 0.6019124984741211),
 ('role', 0.5825510025024414),
 ('accent', 0.5694158673286438),
 ('acting', 0.567068338394165)]

In [19]:
import numpy as np

In [20]:
def avg_word2vec(doc):
    
    return np.mean([model.wv[word] for word in doc if word in model.wv.index_to_key],axis=0)

In [21]:
X=[]
for i in range(len(words)):
    X.append(avg_word2vec(words[i]))

In [22]:
X_new=np.array(X)

In [23]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_new, y, test_size = 0.20, random_state = 0)

In [24]:
from sklearn.ensemble import RandomForestClassifier
classifier=RandomForestClassifier()
classifier.fit(X_train,y_train)

RandomForestClassifier()

In [25]:
y_pred=classifier.predict(X_test)

In [26]:
from sklearn.metrics import accuracy_score,classification_report

In [27]:
print(accuracy_score(y_pred,y_test))
print(classification_report(y_pred,y_test))

0.7847804223935635
              precision    recall  f1-score   support

           0       0.80      0.78      0.79      1564
           1       0.77      0.79      0.78      1419

    accuracy                           0.78      2983
   macro avg       0.78      0.78      0.78      2983
weighted avg       0.79      0.78      0.78      2983

